<a href="https://colab.research.google.com/github/tguardi/clearml/blob/main/Assignment1_Data_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLOps

**Assignment 1: Data Versioning using clear.ml**

+ **Andrew Leonard**
+ **Lev Tyomkin**
+ **Tom Guardi**
+ **Nic Carlson**

**Due date: Thursday, Apr 22, 2021: 6pm**

**Objective:** Leverage clear.ml versioning platform using popular ML dataset

#  Step 1: Import and make dataset dirty

Import libraries

In [33]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle

b) Read in the data

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# enter your drive here
#path = "/content/drive/MyDrive/3 MScA/16. MLOps/Assignment1Data/2019.csv"
path = "/content/drive/MyDrive/Assignment1Data/2019.csv"

In [9]:
df = pd.read_csv(path)

In [10]:
df.head()

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Overall rank                  156 non-null    int64  
 1   Country or region             156 non-null    object 
 2   Score                         156 non-null    float64
 3   GDP per capita                156 non-null    float64
 4   Social support                156 non-null    float64
 5   Healthy life expectancy       156 non-null    float64
 6   Freedom to make life choices  156 non-null    float64
 7   Generosity                    156 non-null    float64
 8   Perceptions of corruption     156 non-null    float64
dtypes: float64(7), int64(1), object(1)
memory usage: 11.1+ KB


Make data dirty

In [12]:
dfDirty = df.copy()
dfDirty.iloc[0] = dfDirty.iloc[0]*-1
dfDirty.head(5)

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,-1,,-7.769,-1.340,-1.587,-0.986,-0.596,-0.153,-0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298


# Step 2: Initialize clear.ml project with dirty data, model, and version

Start using clear.ml with the dirty dataset and version the dataset.

In [15]:
# !pip install clearml

     |████████████████████████████████| 890kB 12.6MB/s 
     |████████████████████████████████| 92kB 6.9MB/s 


# Input credentials from Nic's workspace

In [16]:
!clearml-init

ClearML SDK setup process

Please create new clearml credentials through the profile page in your clearml web app (e.g. http://localhost:8080/profile) 
Or with the free hosted service at https://app.community.clear.ml/profile
In the profile page, press "Create new credentials", then press "Copy to clipboard".

Paste copied configuration here:
{     # Thomas Guardi's workspace     web_server: https://app.community.clear.ml     api_server: https://api.community.clear.ml     files_server: https://files.community.clear.ml     credentials {         "access_key" = "C1P0T34VWHIXJ41ZM8T7"         "secret_key" = "ru2QWf8jSAerRB=JS91Ok$TAfqn+4Jznt6S!24IZ8bP%jIZk+9"     }

Could not parse credentials, please try entering them manually.
Enter user access key: C1P0T34VWHIXJ41ZM8T7
Enter user secret: ru2QWf8jSAerRB=JS91Ok$TAfqn+4Jznt6S!24IZ8bP%jIZk+9
Detected credentials key="C1P0T34VWHIXJ41ZM8T7" secret="ru2Q***"

Editing configuration file: /root/clearml.conf
Enter the url of the clearml-server's 

In [19]:
from clearml import Task
task = Task.init(project_name="assignment1", task_name="create_datasets")

ClearML Task: created new task id=5670f78e38ef410792184ea3abf4b729
ClearML results page: https://app.community.clear.ml/projects/16c95ffffed9471a9e4fa9bdc4582681/experiments/5670f78e38ef410792184ea3abf4b729/output/log


Save dirty dataset to local data folder

In [20]:
dfDirty.to_csv("/content/drive/MyDrive/Assignment1Data/dirty_data.csv")

Create clearml dirty dataset

In [36]:
# Do clear.ml magic e.g. version
!clearml-data create --project assignment1 --name 'dirty_dataset'
!clearml-data sync --folder /content/drive/MyDrive/Assignment1Data/dirty_data.csv

clearml-data - Dataset Management & Versioning CLI
Creating a new dataset:
New dataset created id=b758ecb7d7b043a095c7c92d1db99d57
clearml-data - Dataset Management & Versioning CLI
Syncing dataset id b758ecb7d7b043a095c7c92d1db99d57 to local folder /content/drive/MyDrive/Assignment1Data/dirty_data.csv
Sync completed: 0 files removed, 1 added / modified
Finalizing dataset
Pending uploads, starting dataset upload to https://files.community.clear.ml
Uploading compressed dataset changes (1 files, total 4.9 KB) to https://files.community.clear.ml
Upload completed (4.9 KB)
2021-04-16 19:18:19,075 - clearml.Task - INFO - Waiting to finish uploads
2021-04-16 19:18:19,471 - clearml.Task - INFO - Finished uploading
Dataset closed and finalized


Reload dataset from clearml

In [37]:
from clearml import Dataset

dataset = Dataset.get(dataset_project='assignment1', dataset_name='dirty_dataset')

In [38]:
dataset_folder = dataset.get_mutable_local_copy(target_folder = '/content/drive/MyDrive/working_dataset', overwrite=True)

In [39]:
data = pd.read_csv("/content/drive/MyDrive/working_dataset/dirty_data.csv")

Train Test Split

__Target Variable: Healthy Life Expectancy__

In [40]:
# predicdtors
X = dfDirty[['GDP per capita',
             'Social support',
             'Freedom to make life choices', 
             'Generosity', 
             'Perceptions of corruption']]
# target
y = dfDirty['Healthy life expectancy']

# train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=426)

# upload processed data
# print('Uploading process dataset')
# task.upload_artifact('X_train', X_train)
# task.upload_artifact('X_test', X_test)
# task.upload_artifact('y_train', y_train)
# task.upload_artifact('y_test', y_test)
# store the dataset split into a pickle file
with open(dataset_folder + '/dirty_train.pkl', 'wb') as f:
    pickle.dump([X_train, X_test, y_train, y_test], f)

new_dataset = Dataset.create(
    dataset_project='assignment1', dataset_name='dirty_data_split', parent_datasets=[dataset])
new_dataset.sync_folder(local_path=dataset_folder)
new_dataset.upload()
new_dataset.finalize()

Generating SHA2 hash for 2 files
Hash generation completed
Uploading compressed dataset changes (1 files, total 4.74 KB) to https://files.community.clear.ml
Upload completed (4.74 KB)
2021-04-16 19:18:39,716 - clearml.Task - INFO - Waiting to finish uploads
2021-04-16 19:18:40,098 - clearml.Task - INFO - Finished uploading


True

# Build Clean

In [41]:
# Reverse dirtying steps above
dfClean = dfDirty.copy()
dfClean.iloc[0] = dfClean.iloc[0]*-1

In [42]:
dfClean.to_csv(dataset_folder+"/clean_data.csv")

Update the dataset version to go from dirty to clean dataset.

In [43]:
# Do clear.ml magic e.g. version
new_dataset = Dataset.create(dataset_project = 'assignment1', dataset_name = 'clean_dataset', parent_datasets=[dataset])
new_dataset.sync_folder(local_path=dataset_folder)
new_dataset.upload()
new_dataset.finalize()

Generating SHA2 hash for 3 files
Hash generation completed
Uploading compressed dataset changes (2 files, total 9.6 KB) to https://files.community.clear.ml
Upload completed (9.6 KB)
2021-04-16 19:19:13,428 - clearml.Task - INFO - Waiting to finish uploads
2021-04-16 19:19:13,805 - clearml.Task - INFO - Finished uploading


True

In [44]:
# predictors
X = dfClean[['GDP per capita',
             'Social support',
             'Freedom to make life choices', 
             'Generosity', 
             'Perceptions of corruption']]
# target
y = dfClean['Healthy life expectancy']

# train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=426)

with open(dataset_folder + '/clean_train.pkl', 'wb') as f:
    pickle.dump([X_train, X_test, y_train, y_test], f)

new_dataset = Dataset.create(
    dataset_project='assignment1', dataset_name='clean_data_split', parent_datasets=[dataset])
new_dataset.sync_folder(local_path=dataset_folder)
new_dataset.upload()
new_dataset.finalize()

Generating SHA2 hash for 4 files
Hash generation completed
Uploading compressed dataset changes (3 files, total 14.31 KB) to https://files.community.clear.ml
Upload completed (14.31 KB)
2021-04-16 19:22:55,913 - clearml.Task - INFO - Waiting to finish uploads
2021-04-16 19:22:56,297 - clearml.Task - INFO - Finished uploading


True

# Step 4: Transform, model, and version

Transform the clean dataset using aggregation - call this the transformed dataset.

In [45]:
df['Social Generosity'] = df['Social support'] * df['Generosity']

df.head(1)

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Social Generosity
0,1,Finland,7.769,1.34,1.587,0.986,0.596,0.153,0.393,0.242811


In [46]:
df.corr()

,Overall rank,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Social Generosity
Overall rank,1.000000,-0.989096,-0.801947,-0.767465,-0.787411,-0.546606,-0.047993,-0.351959,-0.417994
Score,-0.989096,1.000000,0.793883,0.777058,0.779883,0.566742,0.075824,0.385613,0.454490
GDP per capita,-0.801947,0.793883,1.000000,0.754906,0.835462,0.379079,-0.079662,0.298920,0.296515
Social support,-0.767465,0.777058,0.754906,1.000000,0.719009,0.447333,-0.048126,0.181899,0.425699
Healthy life expectancy,-0.787411,0.779883,0.835462,0.719009,1.000000,0.390395,-0.029511,0.295283,0.325495
Freedom to make life choices,-0.546606,0.566742,0.379079,0.447333,0.390395,1.000000,0.269742,0.438843,0.483154
Generosity,-0.047993,0.075824,-0.079662,-0.048126,-0.029511,0.269742,1.000000,0.326538,0.864530
Perceptions of corruption,-0.351959,0.385613,0.298920,0.181899,0.295283,0.438843,0.326538,1.000000,0.415001
Social Generosity,-0.417994,0.454490,0.296515,0.425699,0.325495,0.483154,0.864530,0.415001,1.000000


In [47]:
df.to_csv(dataset_folder+'/transformed_dataset.csv')

In [48]:
# Do clear.ml magic e.g. version
new_dataset = Dataset.create(dataset_project = 'assignment1', dataset_name = 'transformed_dataset', parent_datasets=[dataset])
new_dataset.sync_folder(local_path=dataset_folder)
new_dataset.upload()
new_dataset.finalize()

Generating SHA2 hash for 5 files
Hash generation completed
Uploading compressed dataset changes (4 files, total 20.01 KB) to https://files.community.clear.ml
Upload completed (20.01 KB)
2021-04-16 19:23:45,292 - clearml.Task - INFO - Waiting to finish uploads
2021-04-16 19:23:45,680 - clearml.Task - INFO - Finished uploading


True

Update the dataset version with the transformed dataset

In [ ]:
# Copy code from above

Build a machine learning model with the transformed dataset.

In [49]:
# predictors
X = df[['GDP per capita',
             'Social support',
             'Freedom to make life choices', 
             'Generosity', 
             'Perceptions of corruption',
             'Social Generosity']]
# target
y = df['Healthy life expectancy']

# train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=426)

with open(dataset_folder + '/transformed_train.pkl', 'wb') as f:
    pickle.dump([X_train, X_test, y_train, y_test], f)

new_dataset = Dataset.create(
    dataset_project='assignment1', dataset_name='transformed_data_split', parent_datasets=[dataset])
new_dataset.sync_folder(local_path=dataset_folder)
new_dataset.upload()
new_dataset.finalize()

Generating SHA2 hash for 6 files
Hash generation completed
Uploading compressed dataset changes (5 files, total 25.96 KB) to https://files.community.clear.ml
Upload completed (25.96 KB)
2021-04-16 19:24:21,492 - clearml.Task - INFO - Waiting to finish uploads
2021-04-16 19:24:21,882 - clearml.Task - INFO - Finished uploading


True

In [52]:
Task.close

<function clearml.task.Task.close>

In [57]:
Task.current_task()